## Test prompt for S0

In [1]:
from constants.abs_paths import AbsDirPath

from processing_pipeline.utilities.data_transformation import load_all_files

In [2]:

df_o = load_all_files(AbsDirPath.O_KEYWORDS_MATCHING)
df_o.shape

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.issue.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.issue_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\google.deepvariant.v1.6.1.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\matched_o\google.deepvariant.v1.6.1.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\

(1601472, 12)

In [3]:
df_o.head()

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,nunique_matched_words,nsimilar
0,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,Given something that might be a URL (or might ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1
1,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Given a URL, look for the corresponding datase...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1
2,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Download to temporary file, then copy to cache...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,2
3,allenai/scispacy/v0.5.5.code_comment.availabil...,avail,avail,A candidate generator for entity linking to a ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,available,"available, Available, AVAILABLE",3,1
4,allenai/scispacy/v0.5.5.code_comment.availabil...,consist,consist,A spacy pipeline component which identifies en...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,consists,"consists, Consists",2,1


In [4]:
df_o['nwords'] = df_o.sentence.str.count(" ") + 1
df_o = df_o[df_o.nwords > 5]


In [5]:
df_o.groupby('sentence').nsimilar.agg("sum").sort_values(ascending=False)

sentence
Copyright 2018 Google LLC. Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met: 1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. 3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS AS IS AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE

In [6]:
df_sample = df_o.groupby('sentence').nsimilar.agg("sum").sort_values(ascending=False).head(200).reset_index().drop(columns="nsimilar")

In [7]:
AbsDirPath.PT_O_KEYWORDS_MATCHING.mkdir(exist_ok=True)
df_sample.to_parquet(AbsDirPath.PT_O_KEYWORDS_MATCHING / "sample.parquet", index=False, engine="pyarrow", compression="snappy")

In [8]:
from processing_pipeline.s0_noise_filtering.NoiseFiltering import NoiseFilteringStage
from cfg.LLMHost import LLMHost

NoiseFilteringStage(hostname=LLMHost.GREEN_LAB, in_dir_override=AbsDirPath.PT_O_KEYWORDS_MATCHING, out_dir_override=AbsDirPath.PT_S0_NOISE_FILTERING, batch_size_override=20, disable_cache=True).execute()


2025-07-15 12:23:49.610 | INFO     | processing_pipeline.model.BaseStage:execute:243 - Executing s0 stage
2025-07-15 12:23:49.629 | INFO     | processing_pipeline.model.BaseStage:_process_in_batches:151 - Processing sample


Stage s0 params:
model_fields: ['to_eliminate', 'reasoning']
hostname: http://localhost:11434
model: model='deepseek-r1:8b' temperature=0.0 format={'properties': {'to_eliminate': {'title': 'To Eliminate', 'type': 'boolean'}, 'reasoning': {'title': 'Reasoning', 'type': 'string'}}, 'required': ['to_eliminate', 'reasoning'], 'title': 'OllamaFormatValidityResponse', 'type': 'object'} base_url='http://localhost:11434'
batch_size: 10
n_threads: 10
disable_cache: False
in_dir: C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\prompt_tuning\matched_o
out_dir: C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\prompt_tuning\s0_noise_filtering




Processing sample in batches of 10: 100%|██████████| 20/20 [01:51<00:00,  5.57s/it]
2025-07-15 12:25:41.038 | INFO     | processing_pipeline.model.BaseStage:_process_in_batches:192 - Processed sample
2025-07-15 12:25:41.048 | INFO     | processing_pipeline.model.BaseStage:execute:261 - File C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\prompt_tuning\matched_o\sample.parquet processed
2025-07-15 12:25:41.049 | INFO     | processing_pipeline.model.BaseStage:execute:265 - Finished s0 stage


In [9]:
out_dir = AbsDirPath.PT_S0_NOISE_FILTERING
df_results = load_all_files(out_dir)
df_results.to_csv(out_dir / "sample.csv")

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\prompt_tuning\s0_noise_filtering\sample.parquet


In [13]:
df_results.head()

,sentence,s0_prompt,s0_to_eliminate,s0_reasoning
0,Copyright 2018 Google LLC. Redistribution and ...,\nYou are an expert in analyzing and categoriz...,True,This content is a standard copyright and redis...
1,Copyright 2017 Google LLC. Redistribution and ...,\nYou are an expert in analyzing and categoriz...,True,This content is a standard copyright and redis...
2,GLIB - Library of useful routines for C progra...,\nYou are an expert in analyzing and categoriz...,True,This content is primarily copyright and licens...
3,GDK - The GIMP Drawing Kit Copyright (C) 1995-...,\nYou are an expert in analyzing and categoriz...,True,The content is primarily copyright and license...
4,"SGI FREE SOFTWARE LICENSE B (Version 2.0, Sept...",\nYou are an expert in analyzing and categoriz...,True,The content is a formal software license agree...


## Test prompt for S1

In [16]:
import pandas as pd

df_merged = pd.merge(df_o, df_results.drop(columns=["s0_prompt"])[df_results.s0_to_eliminate == False], on="sentence", how="right")
df_merged.shape

(2159, 15)

In [17]:
df_merged.head()

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,nunique_matched_words,nsimilar,nwords,s0_to_eliminate,s0_reasoning
0,scverse/scanpy/1.10.2.wiki.deployability.5362d...,log,log,ghly_variable_genes. scanpy.experimental.pp.re...,deployability,wiki,https://scanpy.readthedocs.io/en/stable/commun...,scverse/scanpy/1.10.2,logfile,logfile,1,53,65,False,This content is a list of functions and classe...
1,scverse/scanpy/1.10.2.wiki.safety.5362d00145b3...,log,log,ghly_variable_genes. scanpy.experimental.pp.re...,safety,wiki,https://scanpy.readthedocs.io/en/stable/commun...,scverse/scanpy/1.10.2,logfile,logfile,1,53,65,False,This content is a list of functions and classe...
2,scverse/scanpy/1.10.2.wiki.security.5362d00145...,log,log,ghly_variable_genes. scanpy.experimental.pp.re...,security,wiki,https://scanpy.readthedocs.io/en/stable/commun...,scverse/scanpy/1.10.2,logfile,logfile,1,53,65,False,This content is a list of functions and classe...
3,scverse/scanpy/1.10.2.wiki.testability.5362d00...,log,log,ghly_variable_genes. scanpy.experimental.pp.re...,testability,wiki,https://scanpy.readthedocs.io/en/stable/commun...,scverse/scanpy/1.10.2,logfile,logfile,1,53,65,False,This content is a list of functions and classe...
4,root-project/root/v6-32-06.code_comment.availa...,error,error,FIXME: This class is only here to support the ...,availability,code_comment,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,error,"error, Error, ERROR, ERRor",4,30,36,False,"Although it starts with a FIXME comment, the c..."


In [18]:
df_merged = df_merged.drop_duplicates(["qa", "sentence"])
df_merged.shape

(606, 15)